## Importing Libraries

In [2]:
import pandas as pd
import sklearn as sk
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Reading in Data
-------
The data used was published by the City of New Orleans at [data.nola.gov](https://datadriven.nola.gov/home/), which aims to promote accountability and data-driven development of the New Orleans area. There are four main files used to examine the actions and effects of the NOPD over the 2022 calendar year:

### Calls for Service: 
This dataset reflects incidents that have been reported to the New Orleans Police Department in 2022. Data is provided by Orleans Parish Communication District (OPCD), the administrative office of 9-1-1 for the City of New Orleans.

- `NOPD_Item` - The NOPD unique item number for the incident
- `Type_` - The NOPD Type associated with the call for service
- `TypeText` - The NOPD TypeText associated with the call for service
- `Priority` - The NOPD Priority associated with the call for service. Code 3 is considered the highest priority and is reserved for officer needs assistance. Code 2 are considered "emergency" calls for service. Code 1 are considered "non-emergency" calls for service. Code 0 calls do not require a police presence. Priorities are differentiated further using the letter designation with "A" being the highest priority within that level.
- `InitialType` - The NOPD InitialType associated with the call for service
- `InitialTypeText` - The NOPD InitialTypeText associated with the call for service
- `InitialPriority` - The NOPD InitialPriority associated with the call for service. See `Priority` description for more information
- `MapX` - The NOPD MapX associated with the call for service. This is provided in state plane and obscured to protect the sensitivity of the data
- `MapY` - The NOPD MapY associated with the call for service. This is provided in state plane and obscured to protect the sensitivity of the data
- `TimeCreate` - The NOPD TimeCreate associated with the call for service. This is the time stamp of th ecreate time of the incident in the CAD system
- `TimeDispatch` - The NOPD TimeDispatch associated with the call for service. This is the entered time by OPCD or NOPD when an officer was dispatched
- `TimeArrive` - The NOPD TimeArrive associated with the call for service. This is the entered time by OPCD or NOPD when an officer arrived
- `TimeClosed` - The NOPD TimeCLosed associated with the call for service. This is the time stamp of the time the call was closed in the CAD system
- `Disposition` - The NOPD Disposition associated with the call for service
- `DispositionText` - The NOPD DispositionText associated with the call for service
- `SelfInitiated` - The NOPD SelfInitiated associated with the call for service. A call is considered self-initiated if the Officer generates the item in the field as opposed to responding to a 911 call
- `Beat` - The NOPD Beat associated with the call for service. This is the area within Orleans Parish that the call for service occurred. The first number is the NOPD District, the letter is the zone, and the numbers are the subzone
- `Block` - The BLOCK unique address number for the incident. The block address has been obscured to protect the sensitivity of the data
- `Zip` - The NOPD Zip associated with the call for service
- `PoliceDistrict` - The NOPD PoliceDistrict associated with the call for service
- `Location` - The NOPD Location associated with the call for service. The X,Y coordinates for the call for service obscured to protect the sensitivity of the data

In [5]:
service_calls = pd.read_csv('../data/Calls_for_Service_2022.csv')
electronic_report = pd.read_csv('../data/Electronic_Police_Report_2022.csv')
misconduct_complaints = pd.read_csv('../data/NOPD_Misconduct_Complaints.csv')
use_of_force = pd.read_csv('../data/NOPD_Use_of_Force_Incidents.csv')